# Imports

In [1]:
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import clear_output

import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from visualkeras import layered_view
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.datasets import mnist, cifar10, fashion_mnist
from tensorflow.keras.layers import Dense, Flatten, UpSampling2D, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.applications import ResNet50, ResNet101, ResNet152

from gand.data import data
from gand.config import MLConfig
from gand.preprocessing import utils
from gand.visualisation import visualise
from gand.models import models, architecture

import pandas as pd
from tabulate import tabulate
from tqdm.keras import TqdmCallback
from sklearn.metrics import classification_report

In [2]:
%load_ext autoreload
%autoreload 2

# Hyperparameters

In [3]:
dataset = cifar10
EPOCHS = 20
BATCH_SIZE = MLConfig.BATCH_SIZE

loss = "categorical_crossentropy"
opt = tf.keras.optimizers.legacy.SGD(learning_rate=0.0001, momentum=0.9)
metrics = ["accuracy"]

# g_model_mnist = load_model(Path.cwd() / 'notebooks/models/cgan/mnist/gen_model_e-200.h5')
# g_model_fashion_mnist = load_model(Path.cwd() / 'notebooks/models/cgan/fashion_mnist/gen_model_e-536.h5')
g_model_cifar10 = load_model(Path.cwd() / 'notebooks/models/cgan/cifar10/gen_model_e-553.h5')
g_model = g_model_cifar10

train_with_gan = True
imbalance_data = True

In [4]:
MLConfig.TYPE_NAMES = ['gans-ResNet50']
                       # 'gans-ResNet50','imbalanced-ResNet50',
                       # 'normal-ResNet101','gans-ResNet101','imbalanced-ResNet101',
                       # 'normal-ResNet152','gans-ResNet152','imbalanced-ResNet152']

# Data Processing

## Normal Data

In [5]:
((X_train_real, y_train_real), (X_test_real, y_test_real)), dataset_name = data.load_dataset(dataset, return_name=True)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

X_train_real = tf.keras.applications.resnet50.preprocess_input(X_train_real)
X_test_real = tf.keras.applications.resnet50.preprocess_input(X_test_real)
print(X_train_real.min(), X_train_real.max(), X_test_real.min(), X_test_real.max())

y_train_real = to_categorical(y_train_real, num_classes=10)
y_test_real = to_categorical(y_test_real, num_classes=10)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

if imbalance_data:
    # REMOVING 50%
    keep_per_class = (X_train_real.shape[0] // (2 * 10))
    X_train_removed, y_train_removed = [], []
    for i in range(y_train_real.shape[-1]):
        class_idx = np.where(np.argmax(y_train_real, axis=-1) == i)[0]
        selected_idx = np.random.choice(class_idx, keep_per_class)
        X_train_removed.append(X_train_real[selected_idx])
        y_train_removed.append(y_train_real[selected_idx])
    
    X_train_real, y_train_real = np.concatenate(X_train_removed, axis=0), np.concatenate(y_train_removed, axis=0)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1) cifar10
-123.68 151.061 -123.68 151.061
(50000, 32, 32, 3) (50000, 10) (10000, 32, 32, 3) (10000, 10) cifar10
(25000, 32, 32, 3) (25000, 10) (10000, 32, 32, 3) (10000, 10) cifar10


## GAN Data

In [6]:
X_train_gan, y_train_gan = None, None
if train_with_gan:
    imbalance_data = True
    # GAN DATA
    n = 3000
    X_train_gan, y_train_gan = data.generate_fake_data(n=n, g_model=g_model, seed=10, verbose=1)
    print(X_train_gan.shape, y_train_gan.shape)
    
    X_train_gan = (X_train_gan+1) * 127.5
    
    X_train_gan = tf.keras.applications.resnet50.preprocess_input(X_train_gan)
    y_train_gan = to_categorical(y_train_gan, num_classes=10)
    
    print(X_train_gan.min(), X_train_gan.max(), y_train_gan.shape)

938/938 [==============================] - 27s 29ms/step
(30000, 32, 32, 3) (30000,)
-123.68 151.061 (30000, 10)


### Appending GAN Data

In [7]:
if train_with_gan:
    X_train_real = np.concatenate((X_train_real, X_train_gan), axis=0)
    y_train_real = np.concatenate((y_train_real, y_train_gan), axis=0)
    print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

(55000, 32, 32, 3) (55000, 10) (10000, 32, 32, 3) (10000, 10) cifar10


## Shuffling Data

In [8]:
X_train_real, y_train_real = shuffle(X_train_real, y_train_real)
X_test_real, y_test_real = shuffle(X_test_real, y_test_real)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

(55000, 32, 32, 3) (55000, 10) (10000, 32, 32, 3) (10000, 10) cifar10


## Making the Dataset Faster

In [9]:
train_data = tf.data.Dataset.from_tensor_slices((X_train_real, y_train_real))
test_data = tf.data.Dataset.from_tensor_slices((X_test_real, y_test_real))

train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_data, test_data, dataset_name

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>,
 'cifar10')

# Model Configuration

In [10]:
train_type = 0
MLConfig.TYPE_NAMES[train_type]

'gans-ResNet50'

In [11]:
base_model = ResNet50(weights='imagenet', include_top=False, 
                      input_shape=(224, 224, 3))

input = Input(shape=(32, 32, 3))
layer = UpSampling2D(size=(7, 7))(input)
layer = base_model(layer)
layer = GlobalAveragePooling2D()(layer)
layer = Flatten()(layer)
layer = Dense(1024, activation='relu')(layer)
layer = Dropout(0.25)(layer)
layer = Dense(512, activation='relu')(layer)
layer = Dropout(0.5)(layer)
output = Dense(10, activation='softmax')(layer)

model = Model(input, output)

## Freezing the Layers

In [15]:
base_model.trainable = True
# opt = tf.keras.optimizers.legacy.SGD()
opt = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss=loss, metrics=metrics)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              209817

# Training the Model

In [16]:
EPOCHS, BATCH_SIZE

(20, 128)

In [17]:
history = model.fit(train_data, batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, validation_data=test_data, verbose=0, 
                    callbacks=[TqdmCallback(verbose=1)], shuffle=True)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

KeyboardInterrupt: 

# Evaluating the Model